In [1]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
import plotly.graph_objects as go

In [2]:
rsm_path = 'input/FULL_TNAV__610'

In [3]:
def index_creator(row):
    just,dance = [],[]
    for j,jtem in enumerate(row):
        if jtem != ' ':
            just.append(j)
        else:
            if len(just) > 0:            
                dance.append([just[0],just[-1]+1])
                just= []
    return dance

In [4]:
def df_maker(header,values):
    it = pd.DataFrame()
    for j,column in header.iteritems():
        check_list = [[j,x,index] for index,x in enumerate(column.values) if type(x) is float] 

        if len(check_list) !=0: 
            head = header.loc[0][j]
            meta = header.loc[1][j]

            if type(header.loc[check_list[0][2]][j]) == float:
                name = 'FIELD'
            elif type(header.loc[check_list[0][2]][j]) == str:
                name = [x for x in header.loc[2:][j].tolist() if x][0]

            values.iloc[:,check_list[0][0]] = values.iloc[:,check_list[0][0]].apply(lambda x: x*check_list[0][1])

            it = pd.concat([it,pd.DataFrame([head,meta,name])],axis = 1,ignore_index=True)
        else:
            head = header.loc[0][j]
            meta = header.loc[1][j]
            try:
                name = [x for x in header.loc[2:][j].tolist() if x][0]
            except:
                if 'TIME' in column.values:
                    name = ''
                else:
                    name = 'FIELD'
            it = pd.concat([it,pd.DataFrame([head,meta,name])],axis = 1,ignore_index=True)      

        df = pd.concat([it,values],axis=0).reset_index(drop = True)
        df.columns = df.loc[0].values
    
    return df

In [5]:
x = open(rsm_path, "r")
res,flag = [], 0

SPACE  = 4 # Гипепараметр, количество символов для отступа

for i,item in  tqdm(enumerate(x)):    
    item = item.split('\n')[0] 
    indexes = index_creator(item)
    if 'TIME' in item:        
        flag = i + 4  #    
        reference = index_creator(item)        
    if i < flag: # Обработка header
        reference_row = []
        indexes = reference
        for j,jtem in enumerate(indexes):
            
            if '*' in item[jtem[0]:jtem[1]+SPACE]:    
                value  = item[jtem[0]:jtem[1]+SPACE]
                value = 10**float(value.split('*10**')[1])
            else:
                         
                value = item[jtem[0]:jtem[1]+SPACE]              
                value = ''.join(value.split()) 
            reference_row.append(value)

        res.append(reference_row)
    else: # Обработка values
        row = []
        for j,jtem in enumerate(indexes):

            value = item[jtem[0]:jtem[1]]
            value = ''.join(value.split())
            row.append(value)
       
        res.append(row)
    if flag == i:
        flag = 0

res = [x for x in res if x] 

54565it [00:00, 97070.03it/s]


In [6]:
def df_maker(header,values):
    it = pd.DataFrame()
    for j,column in header.iteritems():
        check_list = [[j,x,index] for index,x in enumerate(column.values) if type(x) is float] 

        if len(check_list) !=0: 
            head = header.loc[0][j]
            meta = header.loc[1][j]

            if type(header.loc[check_list[0][2]][j]) == float:
                name = 'FIELD'
            elif type(header.loc[check_list[0][2]][j]) == str:
                name = [x for x in header.loc[2:][j].tolist() if x][0]

            values.iloc[:,check_list[0][0]] = values.iloc[:,check_list[0][0]].apply(lambda x: x*check_list[0][1])

            it = pd.concat([it,pd.DataFrame([head,meta,name])],axis = 1,ignore_index=True)
        else:
            head = header.loc[0][j]
            meta = header.loc[1][j]
            try:
                name = [x for x in header.loc[2:][j].tolist() if x][0]
            except:
                if 'TIME' in column.values:
                    name = ''
                else:
                    name = 'FIELD'
            it = pd.concat([it,pd.DataFrame([head,meta,name])],axis = 1,ignore_index=True)      

        df = pd.concat([it,values],axis=0).reset_index(drop = True)
        df.columns = df.loc[0].values
    
    return df

In [7]:
LENGHT = 112 -11

final_df = pd.DataFrame()
for i,row in tqdm(pd.DataFrame(res).iterrows()):

    if 'SUMMARY' in row.values:        
        header = pd.DataFrame(res).loc[i+1:i+4].reset_index(drop= True)        
        values = pd.DataFrame(res).loc[i+5:i+LENGHT].astype(float).reset_index(drop= True)       
        final_df = pd.concat([final_df, df_maker(header,values)],axis=1)              
    else:
        continue

52430it [00:23, 2271.32it/s]


In [ ]:
final_df.to_csv('final_df.csv',sep=',')

In [8]:
final_df.head()

,TIME,YEARS,DAY,MONTH,YEAR,FWPR,FOPR,FGIR,FGPR,FGOR,...,TIME,WTHPH,WTHPH,WTHPH,WTHPH,WTHPH,WTHPH,WTHPH,None,None
0,TIME,YEARS,DAY,MONTH,YEAR,FWPR,FOPR,FGIR,FGPR,FGOR,...,TIME,WTHPH,WTHPH,WTHPH,WTHPH,WTHPH,WTHPH,WTHPH,None,None
1,DAYS,YEARS,DAYS,MONTHS,YEARS,SM3/DAY,SM3/DAY,SM3/DAY,SM3/DAY,SM3/SM3,...,DAYS,BARSA,BARSA,BARSA,BARSA,BARSA,BARSA,BARSA,None,None
2,,FIELD,FIELD,FIELD,FIELD,FIELD,FIELD,FIELD,FIELD,FIELD,...,,3412G,4138G,4561G_1,4136D_G,4571G,4562DG,4503G,FIELD,FIELD
3,0.0,0.0,1.0,7.0,2013.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
4,31.0,0.084932,1.0,8.0,2013.0,0.215343,145.865,0.0,12550.0,86.03613,...,31.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN


In [ ]:
final_df['WOPR']

In [9]:
for i,item in enumerate([['WOPR']]):
    print(item)

    time = final_df.iloc[3:,0].values

    df1 = final_df[[item[0]]]
    #df2 = final_df[[item[1]]]

    fig = go.Figure()
    for j in range(df1.shape[1]):
        fig.add_trace(
            go.Scatter(
            x=time,
            y=df1.iloc[3:,j],
            name=df1.iloc[2,j],
            line=dict(width=1)))

    fig.update_layout(
        title= str(item),
        xaxis_title='Days',
        yaxis_title=df1.loc[1][j],
        width=1400,
        height=600
        )
    fig.show()
    break

['WOPR']
